#### Building A Chatbot

In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This Chatbot
will be able to have conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several
other related concepts that you may be looking for:
* Conversational RAG: Enable a chatbot experience over an external source of data
* Agents: Build a chatbot that can take actions
This only covers the basics

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant",api_key=groq_api_key)
model

v:\Genai_course\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017A99EE0740>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017A9A112510>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is tharun and I am AI Engineer")])

AIMessage(content='Nice to meet you, Tharun. As an AI Engineer, you must be working on exciting projects that involve the development and implementation of artificial intelligence and machine learning models. Can you tell me a bit about the kind of work you do or what areas of AI interest you the most?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 48, 'total_tokens': 107, 'completion_time': 0.088757915, 'prompt_time': 0.002498571, 'queue_time': 0.045866539, 'total_time': 0.091256486}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6ced1025-a807-4802-ba9f-d7780c481c8e-0', usage_metadata={'input_tokens': 48, 'output_tokens': 59, 'total_tokens': 107})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is tharun and I am AI Engineer"),
        AIMessage(content="Hi Tharun, nice to meet you. As an AI Engineer, I'm sure you must be working on some exciting projects. What areas of AI are you particularly interested in or have you worked on? Have you dabbled in deep learning, natural language processing, computer vision, or something else?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)


AIMessage(content='Your name is Tharun and you are an AI Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 131, 'total_tokens': 145, 'completion_time': 0.020903422, 'prompt_time': 0.007348009, 'queue_time': 0.044968176, 'total_time': 0.028251431}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2a3163e5-f2f1-429f-aea6-46274752e214-0', usage_metadata={'input_tokens': 131, 'output_tokens': 14, 'total_tokens': 145})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [6]:
config = {"configurable": {"session_id":"chat1"}}

In [7]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is tharun and I am AI Engineer"),
    ],
    config=config
)

In [8]:
response.content

'Nice to meet you, Tharun. As an AI Engineer, you must be working with various AI technologies and tools. What specific areas of AI are you interested in or working with? Are you into machine learning, natural language processing, computer vision, or something else?'

In [9]:
with_message_history.invoke(
    [
        HumanMessage(content="What's My name"),
    ],
    config=config
)

AIMessage(content='Your name is Tharun.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 117, 'total_tokens': 125, 'completion_time': 0.009521104, 'prompt_time': 0.007953619, 'queue_time': 0.045055217, 'total_time': 0.017474723}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--34e4661a-18ee-40df-94e8-24a3f1218b1d-0', usage_metadata={'input_tokens': 117, 'output_tokens': 8, 'total_tokens': 125})

In [10]:
### Change the config
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

"I don't have any information about your name. This is the start of our conversation, so I don't have any prior knowledge about you. If you'd like to share your name, I'd be happy to know it!"

In [11]:
### Change the config
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="My name is John")],
    config=config1
)
response.content

"Hello John. It's nice to meet you. Is there something I can help you with, or would you like to chat about something in particular?"

In [12]:
### Change the config
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name and can provide some info about Langserve?")],
    config=config1
)
response.content

'Your name is John.\n\nLangServ or LangServe is unclear but one option is LangServ it seems to be a discontinued  service from Microsoft for the task of machine translation.'

#### Prompt Templates
prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","Your are a helpful assistant. Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi, My name is tharun")]})

AIMessage(content='Nice to meet you, Tharun. Is there something I can help you with today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 60, 'total_tokens': 80, 'completion_time': 0.029940968, 'prompt_time': 0.003141347, 'queue_time': 0.044940159, 'total_time': 0.033082315}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--79dc9a5e-489d-4a15-87b3-2d1cbd898fd8-0', usage_metadata={'input_tokens': 60, 'output_tokens': 20, 'total_tokens': 80})

In [15]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)


In [16]:
config = {"configurable":{"session_id":"chat3"}}
response =with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config
)
response.content

"I'm happy to help, but you haven't told me your name. I'm a conversational AI, and our conversation just started, so I don't have any prior information about you. Would you like to share your name with me?"

In [17]:
## Adding more complexity to the prompt

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","Your are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [18]:
response = chain.invoke({"messages":[HumanMessage(content="Hi, My name is tharun")],"language":"Hindi"})
response.content

'नमस्ते थारुन जी, मैं आपकी सहायता करने के लिए तैयार हूँ। कृपया अपने प्रश्न पूछें ताकि मैं आपकी सहायता कर सकूँ।'

Let's now wrap this more complicated chain in a Message History class. This time, because there are mutliple keys in the input, we need to specify the correct key to use to save the chat history.

In [19]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [24]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, My name is tharun")], "language":"Telugu"},
    config=config
)
response.content

'హలో థారున్\u200cగారు! మీ పేరు మళ్లీ వచ్చింది! మీతో మాట్లాడుకోవడానికి నేను ఇక్కడున్నాను. ఏమిటి మాకు తెలియజేయాలి?'

#### Managing the conversation history
one important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [30]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=100,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=True,
    start_on="human"
)

messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi, My name is bob"),
    AIMessage(content="Hi Bob, nice to meet you. "),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="Vanilla is great!"),
    HumanMessage(content="What's 2 + 2?"),
    AIMessage(content="2 + 2 is 4."),
    HumanMessage(content="Thanks!"),
    AIMessage(content="You're welcome! If you have any more questions, feel free to ask."),
    HumanMessage(content="having fun?"),
    AIMessage(content="Absolutely! I'm here to help you with anything you need."),
]
trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, My name is bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi Bob, nice to meet you. ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Vanilla is great!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's 2 + 2?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='2 + 2 is 4.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="You're welcome! If you have any more questions, feel free to ask.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Absolutely! I'm here to help you with anything you need.", additio

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="what ice cream do I like?")],
        "language":"English"
    }
)
response.content

'You mentioned earlier that you like vanilla ice cream.'

In [32]:
### Let's warp this in the message history
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
config = {"configurable":{"session_id":"chat5"}}
response = with_message_history.invoke(
    {'messages':messages + [HumanMessage(content="what my name?")], "language":"English"},
    config=config
)
response.content

"I don't know your name. We just started chatting, and I didn't have any information about you. Would you like to introduce yourself?"